## Serve trained model on your local machine

### Prepare infrastructure in minikube

#### ensure starting from scratch

In [37]:
!minikube delete --all --purge


🔥  Deleting "minikube" in docker ...
🔥  Removing /home/dwarszawski/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.
🔥  Successfully deleted all profiles
💀  Successfully purged minikube directory located at - [/home/dwarszawski/.minikube]


#### check minikube status

In [38]:
!minikube status

🤷  There is no local cluster named "minikube"
👉  To fix this, run: "minikube start"


#### start minikube

In [39]:
!minikube start --kubernetes-version v1.17.0

😄  minikube v1.11.0 on Ubuntu 20.04
✨  Automatically selected the docker driver
🆕  Kubernetes 1.18.3 is now available. If you would like to upgrade, specify: --kubernetes-version=v1.18.3
👍  Starting control plane node minikube in cluster minikube
🎉  minikube 1.14.2 is available! Download it: https://github.com/kubernetes/minikube/releases/tag/v1.14.2
💡  To disable this notice, run: 'minikube config set WantUpdateNotification false'

💾  Downloading Kubernetes v1.17.0 preload ...
    > preloaded-images-k8s-v3-v1.17.0-docker-overlay2-amd64.tar.lz4: 522.40 MiB
🔥  Creating docker container (CPUs=2, Memory=7900MB) ...
🐳  Preparing Kubernetes v1.17.0 on Docker 19.03.2 ...
    ▪ kubeadm.pod-network-cidr=10.244.0.0/16
🔎  Verifying Kubernetes components...
🌟  Enabled addons: default-storageclass, storage-provisioner
🏄  Done! kubectl is now configured to use "minikube"

❗  /usr/local/bin/kubectl is version 1.19.0, which may be incompatible with Kubernetes 1.17.0.
💡  You can also use 'minikube kub

In [28]:
!minikube status


minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured



#### switch kubectl context to use minikube cluster

In [29]:
!kubectl config current-context

minikube


#### create namespace to be used for installing seldon operator

In [40]:
!helm list  --namespace seldon

NAME	NAMESPACE	REVISION	UPDATED	STATUS	CHART	APP VERSION


In [41]:
!kubectl create namespace seldon


namespace/seldon created


In [42]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set ambassador.enabled=true \
    --version v1.2.3 \
    --namespace seldon

NAME: seldon-core
LAST DEPLOYED: Wed Nov 11 13:53:57 2020
NAMESPACE: seldon
STATUS: deployed
REVISION: 1
TEST SUITE: None


#### Enable ambassador in minikube

In [43]:
!minikube addons enable ambassador

🌟  The 'ambassador' addon is enabled


#### Required by Ambassador to expose endpoint

In [ ]:
#!minikube tunnel

#### Secrets to authenticate for artifacts available in GCP storage

In [44]:
#!gcloud auth activate-service-account --key-file spark-bigdl-845ec4674852.json
!kubectl create secret generic user-gcp-sa --namespace seldon --from-file=gcloud-application-credentials.json=spark-bigdl-845ec4674852.json

secret/user-gcp-sa created


In [ ]:
#!kubectl delete secret user-gcp-sa --namespace seldon

#### Service account with secrets assigned

In [45]:
!kubectl apply -f service-account.yaml --namespace seldon



serviceaccount/user-gcp-sa created


In [ ]:
#!kubectl delete sa user-gcp-sa --namespace seldon

### Serve your model and explainer with REST API

In [49]:
!kubectl apply -f seldon-deployment.yaml

seldondeployment.machinelearning.seldon.io/dssconf2020 configured


#### Check ambassador endpoint

In [48]:
!kubectl get services --namespace ambassador

NAME                          TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)                      AGE
ambassador                    LoadBalancer   10.96.139.246   10.96.139.246   80:30947/TCP,443:31277/TCP   5s
ambassador-admin              ClusterIP      10.96.136.65    <none>          8877/TCP                     5s
ambassador-operator-metrics   ClusterIP      10.96.87.16     <none>          8383/TCP,8686/TCP            15s


#### Verify if prediction endpoint exposed with sample input

In [21]:
#!curl -v localhost:9000/api/v1.0/predictions -H "Content-Type: application/json" -d '{"data": {"ndarray": [[4.0, 0.166, 0.986, 0.0, 2.0, 0.0, 1.0]]}}'

!curl http://10.96.24.12/seldon/seldon/dssconf2020/api/v1.0/predictions -H \
    "Content-Type: application/json" \
    -d '{"data": {"ndarray": [[17.75, -0.89100584, 0.45399184, 0.0, 5.0, 1.0, 1.0]]}}'

{"data":{"names":[],"ndarray":[0.014660171157941985]},"meta":{}}


#### Get explanation for prediction

In [22]:
!curl  http://10.96.24.12/seldon/seldon/dssconf2020-explainer/default/api/v1.0/explain \
    -H "Content-Type: application/json" \
    -d '{"data": {"ndarray": [[17.75, -0.89100584, 0.45399184, 0.0, 5.0, 1.0, 1.0]]}}' | \
    jq  '.'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   170  100    93  100    77   6642   5500 --:--:-- --:--:-- --:--:-- 12142
parse error: Invalid numeric literal at line 1, column 17


In [ ]:
!kubectl get pods --namespace seldon

In [ ]:
!kubectl get pods --namespace seldo

In [ ]:
!helm delete seldon-core --namespace seldon
!kubectl delete crds --all


In [ ]:
!kubectl get pods --namespace seldo

#### Clean up minikube

In [ ]:
!helm delete seldon-core --namespace seldon
!kubectl delete crds --all